[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/blog/blob/master/117_langchain/117_langchain.ipynb)

# LangChain 🦜🔗

Con el despegue de los modelos de lenguaje que estamos viviendo en este momento cientos de nuevas herramientas y aplicaciones están apareciendo para aprovechar el poder de estas redes neuronales. Una de ellas parece destacar por encima del resto, y ésta es [LangChain](https://docs.langchain.com/docs/). En este post vamos a ver qué es y cómo podemos usarla.

## ¿Qué es LangChain?

Según su [documentación](https://docs.langchain.com/docs/), Langchain es un entorno de desarrollo de aplicaciones basadas en modelos de lenguajes. Las herramientas proporcionadas por LangChain permiten, por un lado, conectar modelos de lenguaje con otras fuentes de datos (como por ejemplo tus porpios documentos, bases de datos o emails) y, por otro lado, permitir a estos modelos interactuar con su entorno (por ejemplo, enviando emails o llamando a APIs web). Langchain ofrece librerías en Python y Javascript para facilitar el desarrollo de estas aplicaciones, en este post nos centraremos en la librería de Python.

## Un ejemplo práctico

Empezaremos viendo un ejemplo práctico de cómo usar LangChain para proporcionar información sobre un documento, y luego entraremos en detalle de los diferentes componentes y cómo funcionan.

> Vamos a usar como documento el artículo [On the Measure of Intelligence](https://arxiv.org/pdf/1911.01547.pdf), de François Chollet (2019).

Lo primero que necesitamos es instalar la librería de LangChain:

```bash
pip install langchain
````

In [1]:
import langchain

langchain.__version__

'0.0.160'

Primero cargamos el documento

In [2]:
from glob import glob

pdfs = glob('*.pdf')
pdfs

['1911.01547.pdf']

In [3]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("https://arxiv.org/pdf/1911.01547.pdf")
pages = loader.load_and_split()
pages[0]

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


Document(page_content='9 1 0 2\n\nv o N 5 2\n\n] I\n\nA . s c [\n\n2 v 7 4 5 1 0 . 1 1 9 1 : v i X r a\n\nOn the Measure of Intelligence\n\nFranc¸ois Chollet ∗ Google, Inc. fchollet@google.com\n\nNovember 5, 2019\n\nAbstract\n\nTo make deliberate progress towards more intelligent and more human-like artiﬁcial systems, we need to be following an appropriate feedback signal: we need to be able to deﬁne and evaluate intelligence in a way that enables comparisons between two systems, as well as comparisons with humans. Over the past hundred years, there has been an abun- dance of attempts to deﬁne and measure intelligence, across both the ﬁelds of psychology and AI. We summarize and critically assess these deﬁnitions and evaluation approaches, while making apparent the two historical conceptions of intelligence that have implicitly guided them. We note that in practice, the contemporary AI community still gravitates to- wards benchmarking intelligence by comparing the skill exhibited by AI

In [4]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [5]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(model_id="bigscience/bloom-1b7", task="text-generation", model_kwargs={"temperature":0, "max_length":64})


Device has 2 GPUs available. Provide device={deviceId} to `from_model_id` to use availableGPUs for execution. deviceId is -1 (default) for CPU and can be a positive integer associated with CUDA device id.


In [6]:
from langchain import PromptTemplate,  LLMChain

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What is electroencephalography?"

print(llm_chain.run(question))

/home/juan/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 First, we need to understand what is an electroencephalogram. An electroencephalogram is a recording of brain activity. It is a recording of brain activity that is made by placing electrodes on the scalp. The electrodes are placed


## Componentes

- [Data loading](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html)